# Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import librosa
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

## 顯示音頻訊號圖

In [ ]:
path = '/kaggle/input/recoding-data/UAV Recordings 1s/UAV Recordings 1s/Healthy/recording_2023-03-02_14-07-33_012.wav'

# 讀取音頻文件

plt.figure(figsize=(14,5))
data, sampling_rate = librosa.load(path)
librosa.display.waveshow(data,sr=sampling_rate)

## 顯示STFT特徵圖

In [ ]:
# 提取STFT特徵
mfccs_features = librosa.feature.mfcc(y = data, sr = sampling_rate, n_mfcc = 80)

# 繪製STFT
plt.figure(figsize=(10, 4))
librosa.display.specshow(mfccs_features, x_axis='time', sr=sampling_rate)
plt.colorbar(format='%+2.0f dB')
plt.title('MFCC')
plt.tight_layout()
plt.show()

## 加載音頻訊號

In [ ]:
data_path = '/kaggle/input/recoding-data/UAV Recordings 1s/UAV Recordings 1s'

## 音頻訊號進行STFT特徵提取

In [ ]:
def load_data(data_dir):
    datasets = []
    labels = []
    categories = os.listdir(data_dir)
    for index, folder in enumerate(categories):
        for filename in os.listdir(data_dir+"/"+folder):
            data, sampling_rate = librosa.load(data_dir+"/"+folder+"/"+filename)
            stft_feature = np.abs(librosa.stft(data))
            scaled_features = np.mean(stft_feature.T,axis=0)
            datasets.append(np.array(scaled_features))
            labels.append(categories[index])
    return np.array(datasets), np.array(labels)

datasets, labels = load_data(data_path)

In [ ]:
datasets = datasets.astype('float32')

labelencoder = LabelEncoder()

labelsOneHot = to_categorical(labelencoder.fit_transform(labels))

## 切分成訓練資料、測試資料，8:2的比例

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(datasets, labelsOneHot, test_size = 0.2, stratify = labelsOneHot, random_state = 3)

print("----------Train-------------")
print(X_train.shape)

print("----------Test--------------")
print(X_test.shape)

## 類別數量

In [ ]:
num_classes = len(np.unique(labels))

## 建立NN模型

In [ ]:
model=Sequential()

#Input layer
model.add(Flatten(input_shape=(datasets.shape[1],)))

#hidden layer 1
model.add(Dense(512, Activation('relu')))

#hidden layer 2
model.add(Dense(128, Activation('relu')))

#hidden layer 3
model.add(Dense(64, Activation('relu')))

# 使用Dropout防止過度擬合
model.add(Dropout(0.25))

#Output layer
model.add(Dense(num_classes, Activation('softmax')))

In [ ]:
# 選擇損失函數、優化方法及成效衡量方式
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## 模型訓練

In [ ]:
train_history = model.fit(X_train, y_train, validation_split=0.2, batch_size = 100, epochs = 150, verbose=2)

## 評估訓練結果

In [ ]:
def show_train_history(train_history, train, validation):

    plt.plot(train_history.history[train])

    plt.plot(train_history.history[validation])

    plt.title('Train History')

    plt.ylabel('train')

    plt.xlabel('Epoch')

    plt.legend(['train', 'validation'], loc='upper left')

    plt.show()
    

show_train_history(train_history, 'accuracy', 'val_accuracy')
show_train_history(train_history, 'loss', 'val_loss')

## 使用未參與訓練的測試資料測試，模型準確率

In [ ]:
scores = model.evaluate(X_test,y_test,verbose=0)

print('Accuracy = ', scores[1])

## 評估訓練結果

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)
confusion_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10,7))
sns.heatmap(confusion_matrix, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()